### Baseline F1-scores for cross-corpus binary classification ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_popane as dr_p
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\zhoux\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_apd = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", 
        "Speech_Relax", "Speech_Anticipate"
    ],
    # [
    #     "Baseline_Rest", 
    #     "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
    #     "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    # ],
    ["BugBox_Break", "Speech_Break"],
    ["BugBox_Exposure", "Speech_Exposure"]
]

model_phases_wesad = [
    [
        dr_w.Phases.BASE,
        dr_w.Phases.FUN,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
    ]
]

studies_popane = [
    "Study1",
    "Study3",
    # "Study4",
    "Study5"
]

model_phases_popane = [
    dr_p.Study1.ALL,
    dr_p.Study3.ALL,
    # dr_p.Study4.ALL,
    dr_p.Study5.ALL
]

apd_label_type = "Anxiety"
popane_label_type = "affect"
wesad_label_type = "stai"

test_size = 1.0

`use_label_encoder` is deprecated in 1.7.0.


In [7]:
# TRAIN ON APD AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


for i, phases_apd in enumerate(model_phases_apd):
    for j, phases_wesad in enumerate(model_phases_wesad):
        print(f"APD PHASES {i} " + "-"*50)
        print(f"WESAD PHASES {j} " + "-"*50)
        x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type)
        # drop subjects with noisy data
        x_a = x_a[x_a['subject'] != 84.0]
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature
        y_a = y_a[y_a['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]
        
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        x_train_a, y_train_a, x_test_a, y_test_a, test_subjects = train.train_test_split(x_a, y_a, test_size=0.0, by_subject=False)
        x_train_b, y_train_b, x_test_b, y_test_b, test_subjects = train.train_test_split(x_b, y_b, test_size=test_size, by_subject=False)
        # print(f"x_train: {x_train.shape}")
        # print(f"y_train: {y_train.shape}")
        x_train = pd.concat([x_train_a, x_train_b])
        y_train = pd.concat([y_train_a, y_train_b])
        x_test = x_test_b
        y_test = y_test_b.loc[:, "label"]

        y_pred_pos = np.ones(shape=y_test.shape)
        y_pred_neg = np.zeros(shape=y_test.shape)
        y_pred_random = np.asarray([random.choice([0, 1]) for _ in range(y_test.shape[0])])

        p = precision_score(y_test, y_pred_pos, zero_division=1)
        r = recall_score(y_test, y_pred_pos, zero_division=1)
        f1 = f1_score(y_test, y_pred_pos, zero_division=1)
        auc = roc_auc_score(y_test, y_pred_pos)

        print("ALL POSITIVE " + "-"*30)
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        
        p = precision_score(y_test, y_pred_neg, zero_division=1)
        r = recall_score(y_test, y_pred_neg, zero_division=1)
        f1 = f1_score(y_test, y_pred_neg, zero_division=1)
        auc = roc_auc_score(y_test, y_pred_neg)

        print("ALL NEGATIVE " + "-"*30)
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")

        p = precision_score(y_test, y_pred_random, zero_division=1)
        r = recall_score(y_test, y_pred_random, zero_division=1)
        f1 = f1_score(y_test, y_pred_random, zero_division=1)
        auc = roc_auc_score(y_test, y_pred_random)

        print("RANDOM " + "-"*30)
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")

APD PHASES 0 --------------------------------------------------
WESAD PHASES 0 --------------------------------------------------
ALL POSITIVE ------------------------------
Precision: 0.37142857142857144
Recall: 1.0
F1-score: 0.5416666666666666
AUC score: 0.5
ALL NEGATIVE ------------------------------
Precision: 1.0
Recall: 0.0
F1-score: 0.0
AUC score: 0.5
RANDOM ------------------------------
Precision: 0.35135135135135137
Recall: 0.5
F1-score: 0.41269841269841273
AUC score: 0.4772727272727273


APD PHASES 0 --------------------------------------------------
WESAD PHASES 1 --------------------------------------------------
ALL POSITIVE ------------------------------
Precision: 0.39285714285714285
Recall: 1.0
F1-score: 0.5641025641025641
AUC score: 0.5
ALL NEGATIVE ------------------------------
Precision: 1.0
Recall: 0.0
F1-score: 0.0
AUC score: 0.5
RANDOM ------------------------------
Precision: 0.35714285714285715
Recall: 0.45454545454545453
F1-score: 0.4
AUC score: 0.46256684491

In [10]:
# TRAIN ON APD AND TEST ON POPANE
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


for i, phases_apd in enumerate(model_phases_apd):
    for j, study in enumerate(studies_popane):
        print(f"POPANE PHASES {j} " + "-"*50)
        print(f"APD PHASES {i} " + "-"*50)
        x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        x_b, y_b = train.Train_POPANE.get_popane_data(study, metrics, model_phases_popane[j], verbose=False, label_type=popane_label_type)
        inds = pd.isnull(x_b).any(1).to_numpy().nonzero()[0]
        x_b = x_b.drop(inds, axis=0)
        y_b = y_b.drop(inds, axis=0)

        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)
        
        # drop subjects with noisy data
        x_a = x_a[x_a['subject'] != 84.0]
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because POPANE doesn't have this feature
        y_a = y_a[y_a['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        x_train_a, y_train_a, x_test_a, y_test_a, test_subjects = train.train_test_split(x_a, y_a, test_size=0.0, by_subject=False)
        x_train_b, y_train_b, x_test_b, y_test_b, test_subjects = train.train_test_split(x_b, y_b, test_size=test_size, by_subject=False)
        # print(f"x_train: {x_train.shape}")
        # print(f"y_train: {y_train.shape}")
        x_train = pd.concat([x_train_a, x_train_b])
        y_train = pd.concat([y_train_a, y_train_b])
        x_test = x_test_b
        y_test = y_test_b.loc[:, "label"]

        y_pred_pos = np.ones(shape=y_test.shape)
        y_pred_neg = np.zeros(shape=y_test.shape)
        y_pred_random = np.asarray([random.choice([0, 1]) for _ in range(y_test.shape[0])])

        p = precision_score(y_test, y_pred_pos, zero_division=1)
        r = recall_score(y_test, y_pred_pos, zero_division=1)
        f1 = f1_score(y_test, y_pred_pos, zero_division=1)
        auc = roc_auc_score(y_test, y_pred_pos)

        print("ALL POSITIVE " + "-"*30)
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        
        p = precision_score(y_test, y_pred_neg, zero_division=1)
        r = recall_score(y_test, y_pred_neg, zero_division=1)
        f1 = f1_score(y_test, y_pred_neg, zero_division=1)
        auc = roc_auc_score(y_test, y_pred_neg)

        print("ALL NEGATIVE " + "-"*30)
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")

        p = precision_score(y_test, y_pred_random, zero_division=1)
        r = recall_score(y_test, y_pred_random, zero_division=1)
        f1 = f1_score(y_test, y_pred_random, zero_division=1)
        auc = roc_auc_score(y_test, y_pred_random)

        print("RANDOM " + "-"*30)
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")

POPANE PHASES 0 --------------------------------------------------
APD PHASES 0 --------------------------------------------------
ALL POSITIVE ------------------------------
Precision: 0.23756906077348067
Recall: 1.0
F1-score: 0.38392857142857145
AUC score: 0.5
ALL NEGATIVE ------------------------------
Precision: 1.0
Recall: 0.0
F1-score: 0.0
AUC score: 0.5
RANDOM ------------------------------
Precision: 0.22413793103448276
Recall: 0.45348837209302323
F1-score: 0.3
AUC score: 0.4821789686552073


POPANE PHASES 1 --------------------------------------------------
APD PHASES 0 --------------------------------------------------
ALL POSITIVE ------------------------------
Precision: 0.6634382566585957
Recall: 1.0
F1-score: 0.7976710334788938
AUC score: 0.5
ALL NEGATIVE ------------------------------
Precision: 1.0
Recall: 0.0
F1-score: 0.0
AUC score: 0.5
RANDOM ------------------------------
Precision: 0.6619047619047619
Recall: 0.5072992700729927
F1-score: 0.5743801652892562
AUC score